In [1]:
# Load kedro environment (not needed in .py)
from pathlib import Path
from kedro.framework.context import load_context

# Load a context to be able to work in the notebook
current_dir = Path("/u01/share/cesar/aa_engine_cl/notebooks/04_features")
#current_dir = Path.cwd()
proj_path = current_dir.parent.parent
context = load_context(proj_path)
catalog = context.catalog
credentials = context.config_loader.get("credentials*","credentials*/**")
parameters = context.config_loader.get("parameters*","parameters*/**")

In [22]:
date="20210101"
intencion_baja = catalog.load("intencion_baja")
cliente_activo = catalog.load("cliente_activo")

2021-06-09 19:49:04,292 - kedro.io.data_catalog - INFO - Loading data from `intencion_baja` (SQLPartitionedDataSet)...
2021-06-09 19:49:04,294 - kedro.io.data_catalog - INFO - Loading data from `cliente_activo` (SQLPartitionedDataSet)...


In [24]:
cliente_activo_df =  create_cliente_activo(cliente_activo,date)

2021-06-09 19:49:17,844 - aa_engine_pkg.assets.utils.utilities - INFO - Creating cliente_activo...
select distinct CUSTOMER_ID from stg_cl_customer_status where STATUS = 'A - Activo' and DATE_EXP = 202012


/u01/miniconda3/envs/uypo37_2/lib/python3.7/site-packages/sqlalchemy/dialects/oracle/base.py:1381: SAWarning: Oracle version (19, 5, 0, 0, 0) is known to have a maximum identifier length of 128, rather than the historical default of 30. SQLAlchemy 1.4 will use 128 for this database; please set max_identifier_length=128 in create_engine() in order to test the application with this new length, or set to 30 in order to assure that 30 continues to be used.  In particular, pay close attention to the behavior of database migrations as dynamically generated names may change. See the section 'Max Identifier Lengths' in the SQLAlchemy Oracle dialect documentation for background.
  % ((self.server_version_info,))


In [13]:
# Read parameters
log = initialize_logger()

write_to_parquet = parameters["write_to_parquet"]
table_name = "intencion_baja"
overwrite = parameters["masters"][table_name]["overwrite"]

# Check if table was already created
files = get_mastertable_paths_by_period(parameters=parameters, period=date)
match = [str(file) for file in files if table_name in file]

In [31]:
log.info("Creating master table intencion baja")

 # Read parameters
date = pd.to_datetime(date)
look_back_days = parameters["masters"]["global"]["look_back_days"]
start_date = date - timedelta(days=look_back_days)

#   start_date = (pd.to_datetime(date) - timedelta(days=look_back_days)).strftime("%Y%m%d")

#  look_back_days = parameters["masters"]["intencion_baja"]["look_back_days"]
#  start_date = (pd.to_datetime(date) - timedelta(days=look_back_days)).strftime("%Y%m%d")


# Calculate period to load for active clients
period_to_load = get_previous_month(date)
df_clientes = cliente_activo_df

# Get intencion de baja and churn tables
df_ib = intencion_baja.filter_by(date=[start_date.strftime("%Y%m%d"),
                                            date.strftime("%Y%m%d")])

2021-06-09 19:55:05,593 - aa_engine_pkg.assets.utils.utilities - INFO - Creating master table intencion baja
select MES, FECHA, MOVIMIENTO, CANAL, BASIC_PRODUCT_NAME, SHSUBSCRIBERNR, DATE_EXP from stg_cl_intencion_baja where FECHA >= to_date('20200717', 'yyyymmdd') and FECHA < to_date('20210101', 'yyyymmdd')


In [32]:
df_ib.head()

,MES,FECHA,MOVIMIENTO,CANAL,BASIC_PRODUCT_NAME,SHSUBSCRIBERNR,DATE_EXP
0,2020-08-31,2020-08-18,RETENCION,INBOUND,Plan Oro HD,119448257,202008
1,2020-08-31,2020-08-17,RETENCION,INBOUND,Plan Oro HD,119462992,202008
2,2020-08-31,2020-08-17,RETENCION,INBOUND,Plan Oro HD,119463085,202008
3,2020-08-31,2020-08-17,RETENCION,IVR NBO,Plan Oro HD,119465460,202008
4,2020-08-31,2020-08-18,RETENCION,INBOUND,Plan Oro HD,119478858,202008


In [33]:
df_ib.MOVIMIENTO.unique()

array(['RETENCION', 'CANCEL. AGENDADA', 'CANCEL. INMEDIATA',
       'CANCEL. INMEDIATA (DS A C)', 'SUSP. INMEDIATA', 'SUSP. AGENDADA'],
      dtype=object)

In [34]:
df_ib["MOVIMIENTO"] = df_ib.MOVIMIENTO.str.replace(" ","_")
df_ib["CANAL"] = df_ib.CANAL.str.replace(" ","_")
df_ib["BASIC_PRODUCT_NAME"] = df_ib.BASIC_PRODUCT_NAME.str.replace(" ","_") 

In [35]:
# Format
df_ib.rename(columns={"SHSUBSCRIBERNR": "CUSTOMER_ID"}, inplace=True)
df_ib["FECHA"] = df_ib["FECHA"].dt.strftime("%Y%m%d")
df_ib.rename(columns={"BASIC_PRODUCT_NAME": "PRODUCTO_BASICO"}, inplace=True)

In [36]:
df_ib["N_CALL"] = 1

# Merging
df_master_ib = df_clientes.merge(df_ib,
                             on="CUSTOMER_ID",
                             how="inner")


choicelist = ["ORO","PLATA","BRONCE"]
condlist = [df_master_ib['PRODUCTO_BASICO'].str.contains("ORO", case=False, na = False),
df_master_ib['PRODUCTO_BASICO'].str.contains("PLATA", case=False, na = False),
df_master_ib['PRODUCTO_BASICO'].str.contains("BRONCE", case=False, na = False)]

In [37]:
df_master_ib .head(2)

,CUSTOMER_ID,MES,FECHA,MOVIMIENTO,CANAL,PRODUCTO_BASICO,DATE_EXP,N_CALL
0,57068211,2020-10-31,20201008,RETENCION,INBOUND,Pack_ORO_HD,202010,1
1,118851463,2020-10-31,20201021,RETENCION,INBOUND,Plan_Oro_HD,202010,1


In [39]:
df_master_ib["TIPO_PRODUCTO"] = np.select(condlist, choicelist, default='OTROS')


# Create dummy variables
df = []
for var in vars_to_dummy:
    log.info(f'---- {var}')
    df.append(create_dummy_variables(df=df_master_ib,
                             vars_to_groupby=["CUSTOMER_ID", "FECHA"],
                             var_to_dummy= var,
                             include_total=False))

2021-06-09 19:56:21,497 - aa_engine_pkg.assets.utils.utilities - INFO - ---- MOVIMIENTO
2021-06-09 19:56:21,618 - aa_engine_pkg.assets.utils.utilities - INFO - ---- TIPO_PRODUCTO
2021-06-09 19:56:21,691 - aa_engine_pkg.assets.utils.utilities - INFO - ---- CANAL


In [40]:
# Reduce list of list to create a data table
df = reduce(lambda left, right: pd.merge(left, right, on=["CUSTOMER_ID", "FECHA"], how="outer"), df)

# Create expanded variables
df_expanded = add_relative_calculate_past(df=df,
                                  id_cols=["CUSTOMER_ID"],
                                  periods=past_periods,
                                  agg=[np.nansum],
                                  date_col="FECHA",
                                  start_date=start_date,
                                  end_date=date,
                                  period_freq="D")  

2021-06-09 19:56:56,616 - aa_engine_pkg.assets.utils.utilities - INFO - Adding relative date between 2020-07-17 00:00:00 and 2021-01-01 00:00:00
2021-06-09 19:56:56,650 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 7
2021-06-09 19:56:56,710 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 14
2021-06-09 19:56:56,767 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 21
2021-06-09 19:56:56,826 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 28
2021-06-09 19:56:56,885 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 56
2021-06-09 19:56:56,948 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 84
2021-06-09 19:56:57,054 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 112
2021-06-09 19:56:57,200 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 140
2021-06-09 19:56:57,414 - aa_e

In [42]:
df["FECHA"] = pd.to_datetime(df["FECHA"], format="%Y%m%d")
df["DAYS"] = (date - df["FECHA"])/np.timedelta64(1,"D")
df_days = flatten_df(df.groupby("CUSTOMER_ID").agg({"DAYS":[np.nanmin, np.nanmax]}))


df_final = df_expanded.merge(df_days, on="CUSTOMER_ID")

# Add date variables
df_final["DATE_EXP"] = period_to_load
df_final["DATE_CALC"] = datetime.strftime(date, "%Y%m%d")


In [43]:
df_final.head(2)

,CUSTOMER_ID,CANAL_SUPER_OUTBOUND_nansum_7,MOVIMIENTO_RETENCION_nansum_7,MOVIMIENTO_CANCEL._AGENDADA_nansum_7,MOVIMIENTO_SUBTOTAL_nansum_7,MOVIMIENTO_SUSP._INMEDIATA_nansum_7,TIPO_PRODUCTO_ORO_nansum_7,CANAL_INBOUND_nansum_7,CANAL_TLV_UPGRADE_nansum_7,CANAL_IVR_NBO_nansum_7,...,CANAL_SUPER_INBOUND_nansum_168,TIPO_PRODUCTO_OTROS_nansum_168,TIPO_PRODUCTO_PLATA_nansum_168,CANAL_SUBTOTAL_nansum_168,TIPO_PRODUCTO_SUBTOTAL_nansum_168,MOVIMIENTO_CANCEL._INMEDIATA_nansum_168,DAYS_nanmin,DAYS_nanmax,DATE_EXP,DATE_CALC
0,26634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,1.0,1.0,0.0,144.0,144.0,202012,20210101
1,27228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,1.0,1.0,0.0,88.0,88.0,202012,20210101


In [2]:
from aa_engine_pkg.assets.utils import *
from aa_engine_pkg.assets.core.data.kedro.catalog_expansion.partitioned_sql import SQLPartitionedDataSet

PRODUCTO = ["ORO","PLATA","BRONCE"]

vars_to_dummy = ["MOVIMIENTO","TIPO_PRODUCTO","CANAL"]

past_periods = [7, 14, 21, 28, 56, 84, 112, 140, 168]



def create_master_intencion_baja(intencion_baja: SQLPartitionedDataSet,
                                 cliente_activo: pd.DataFrame,
                                 parameters: Union[Dict, None],
                                 date: str) -> pd.DataFrame:
    """Creates master table with unsubscribe intention of client's


    Parameters
    ----------
    intencion_baja:
        dataset defined in ``catalog_raw.yml`` with raw data information related to client's unssuscribe intention
    cliente_activo:
        dataset defined in ``catalog_raw.yml`` with raw data information related to active clients at EoP
    date:
        period to process
    parameters:
        set of project parameters defined in ``parameters.yml``

    Returns
    -------
    pd.DataFrame
        Master table with offer features for one period

    """

    # Read parameters
    log = initialize_logger()

    write_to_parquet = parameters["write_to_parquet"]
    table_name = "intencion_baja"
    overwrite = parameters["masters"][table_name]["overwrite"]

    # Check if table was already created
    files = get_mastertable_paths_by_period(parameters=parameters, period=date)
    match = [str(file) for file in files if table_name in file]

    if len(match) > 0 and overwrite is False:
        # If table is found, read parquet:
        log.info(f"Reading {match[0]} table")
        df_final = pd.read_parquet(match[0], engine="pyarrow")

    else:
       # Initialize logger
        log = initialize_logger()
        log.info("Creating master table intencion baja")

         # Read parameters
        date = pd.to_datetime(date)
        look_back_days = parameters["masters"]["global"]["look_back_days"]
        start_date = date - timedelta(days=look_back_days)

      #   start_date = (pd.to_datetime(date) - timedelta(days=look_back_days)).strftime("%Y%m%d")
        
      #  look_back_days = parameters["masters"]["intencion_baja"]["look_back_days"]
      #  start_date = (pd.to_datetime(date) - timedelta(days=look_back_days)).strftime("%Y%m%d")
       
        
        # Calculate period to load for active clients
        period_to_load = get_previous_month(date)
        df_clientes = cliente_activo

        # Get intencion de baja and churn tables
        df_ib = intencion_baja.filter_by(date=[start_date.strftime("%Y%m%d"),
                                                    date.strftime("%Y%m%d")])

        #df_ib = intencion_baja.filter_by(date=[start_date, date]).drop_duplicates()
        

        df_ib["MOVIMIENTO"] = df_ib.MOVIMIENTO.str.replace(" ","_")
        df_ib["CANAL"] = df_ib.CANAL.str.replace(" ","_")
        df_ib["BASIC_PRODUCT_NAME"] = df_ib.BASIC_PRODUCT_NAME.str.replace(" ","_") 

        # Format
        df_ib.rename(columns={"SHSUBSCRIBERNR": "CUSTOMER_ID"}, inplace=True)
        df_ib["FECHA"] = df_ib["FECHA"].dt.strftime("%Y%m%d")
        df_ib.rename(columns={"BASIC_PRODUCT_NAME": "PRODUCTO_BASICO"}, inplace=True)

        # Count number of calls
        df_ib["N_CALL"] = 1

        # Merging
        df_master_ib = df_clientes.merge(df_ib,
                                         on="CUSTOMER_ID",
                                         how="inner")


        choicelist = ["ORO","PLATA","BRONCE"]
        condlist = [df_master_ib['PRODUCTO_BASICO'].str.contains("ORO", case=False, na = False),
            df_master_ib['PRODUCTO_BASICO'].str.contains("PLATA", case=False, na = False),
            df_master_ib['PRODUCTO_BASICO'].str.contains("BRONCE", case=False, na = False)]
            
            
        df_master_ib["TIPO_PRODUCTO"] = np.select(condlist, choicelist, default='OTROS')


        # Create dummy variables
        df = []
        for var in vars_to_dummy:
            log.info(f'---- {var}')
            df.append(create_dummy_variables(df=df_master_ib,
                                     vars_to_groupby=["CUSTOMER_ID", "FECHA"],
                                     var_to_dummy= var,
                                     include_total=False))

    
    
        # Reduce list of list to create a data table
        df = reduce(lambda left, right: pd.merge(left, right, on=["CUSTOMER_ID", "FECHA"], how="outer"), df)
    
        # Create expanded variables
        df_expanded = add_relative_calculate_past(df=df,
                                          id_cols=["CUSTOMER_ID"],
                                          periods=past_periods,
                                          agg=[np.nansum],
                                          date_col="FECHA",
                                          start_date=start_date,
                                          end_date=date,
                                          period_freq="D")    
    
    
        df["FECHA"] = pd.to_datetime(df["FECHA"], format="%Y%m%d")
        df["DAYS"] = (date - df["FECHA"])/np.timedelta64(1,"D")
        df_days = flatten_df(df.groupby("CUSTOMER_ID").agg({"DAYS":[np.nanmin, np.nanmax]}))


        df_final = df_expanded.merge(df_days, on="CUSTOMER_ID")

        # Add date variables
        df_final["DATE_EXP"] = period_to_load
        df_final["DATE_CALC"] = datetime.strftime(date, "%Y%m%d")

       # Rename table
        rename_table(df=df_final,
             preffix="IBA",
             ids_to_exclude=["CUSTOMER_ID", "DATE_EXP", "DATE_CALC"]
            )

        # Return
        log.info(f"Exporting {df_final.shape[0]} rows and {df_final.shape[1]} columns") 
         
        date_ = date.strftime("%Y%m%d")
        
        if write_to_parquet:
            log.info(f"\n------ Writing {table_name} for period {date} to parquet ------")
            file = f"{parameters['paths']['master_path']}master_{table_name}/master_{table_name}_{date_}.parquet"
            df_final.to_parquet(file, engine="pyarrow")

    return df_final

In [23]:
def create_cliente_activo(cliente_activo: SQLPartitionedDataSet,
                          date: str) -> pd.DataFrame:
    """Creates master table with features related to EoP state of customers for one period of data
    Parameters
    ----------
    cliente_activo:
        dataset defined in ´catalog.yml´ - list of active customers at EoP for the given period
    date:
        period to process
    Returns
    -------
        Mastertable with information of clientes at EoP
    """

    # Initialize logger
    log = initialize_logger()

    # Load active clientes for period
    log.info(f"Creating cliente_activo...")
    period_to_load = get_previous_month(date)
    df_clientes_activos = cliente_activo.filter_by(date=period_to_load)

    return df_clientes_activos